In [6]:
# import numpy
import pandas as pd
# from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('raw_data.csv', encoding='utf-8-sig', index_col=0)

In [3]:
# 행 번호 삭제
df.drop('cpname', axis=1, inplace=True)
df.drop('title', axis=1, inplace=True)
df.drop('career', axis=1, inplace=True)
df.head()

,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros
0,서울,학력무관,정규직,중소기업,"5,342억 8,400만원 (2022)","1,251 명 재직중","6,218만원\n자세히보기",550억원,1000대-안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-매출 1000대 기...
1,서울,고졸,계약직,중소기업,"5,342억 8,400만원 (2022)","1,251 명 재직중",-,550억원,1000대-KOSPI-복지의 끝판왕-매출 1조 클럽-보너스로 행복해요♥-집 걱정 안...
2,서울,대졸이상,정규직,대기업,"18조 3,714억 3,700만원 (2023)","19,824 명 재직중","7,656만원\n자세히보기","1조 5,644억 9,900만원",1000대-KT그룹-안정적인 장수 회사-구직자 인기기업-매출 1조 클럽-HOT한 인...
3,서울,대졸이상,정규직,대기업,"8,477억 5,653만원 (2022)",409 명 재직중,"7,177만원\n자세히보기","677억 1,334만원",1000대-한화그룹-성장판 열린 회사-구직자 인기기업-평균연봉 6천이상-지방에 위치...
4,서울,학력무관,정규직,대기업,"8,477억 5,653만원 (2022)",409 명 재직중,-,"677억 1,334만원",KOSDAQ-구직자 인기기업-복지의 끝판왕-아침을 왜 굶어요?-자기계발비 지원 기업...


In [4]:
tmp = str(df['pros'][5])
list = tmp.split('-')
print(list)
print(len(list))

['우수기업', '구직자 인기기업', '채용에 적극적인 회사', 'HOT한 인기기업', '인크루트 인증 우수기업', '공기업 in 서울']
6


In [5]:
# 다른 피쳐들은 전처리 후에도 값 범위가 클 것이므로 
# 아래 네 피쳐의 label encoding 값에 적절한 크기의 가중치를 곱할 예정
# region : Label encoding	
# education	: Label encoding
# jobtype	: Label encoding
# cptype : Label encoding

# 매출 등의 숫자들은 편차가 너무 크므로 곧바로 min-max scaling 이나 정규화를 적용시키는 것은 부적절하다 판단
# 1. 범위에 따라 적절하게 단위를 변환할 것 (예 : 천만원 단위)
# 2. 매우 큰 값 (수십 조) 가 존재하므로 min-max scaling 대신 정규화 적용
# sales
# aversalary
# employees
# capital 

# pros 
# 1안) 항목의 갯수를 센 후 스케일을 다른 피쳐와 맞출 예정 (가중치가 동일한 단순합 형태)
# 2안) 수동으로 항목마다 가중치를 다르게 해서 가중합 후 스케일을 다른 피쳐와 맞추기 >> 노가다

# ex) 빵빵한 구내식당 - 1점. 매출 1000대 기업 - 8점 -> 1안) 2점 / 2안) 9점

In [6]:
print(df['education'].value_counts())
print(df['jobtype'].value_counts())
print(df['cptype'].value_counts())

education
학력무관     57
대졸이상     18
초대졸이상     6
고졸        3
고졸이상      1
Name: count, dtype: int64
jobtype
정규직     36
정규직외    21
계약직     15
인턴       6
알바       6
계약직외     1
Name: count, dtype: int64
cptype
대기업     59
중견기업    10
중기업      8
중소기업     6
소기업      2
Name: count, dtype: int64


In [7]:
# label encoding
# 나중에 선형모델에서 feature 값들을 바탕으로 '좋은 일자리' 에 대한 점수를 매길 것을 감안하고 label encoding
df['education'].replace({
    '학력무관': 0,
    '고졸': 1,
    '고졸이상': 2,
    '초대졸이상': 3,
    '대졸이상': 4
}, inplace=True)


df['jobtype'].replace({
    '알바' : 0,
    '인턴' : 1,
    '계약직': 2,
    '계약직외' : 2,
    '정규직' : 3,
    '정규직외' : 3
}, inplace=True)

df['cptype'].replace({
    '소기업': 0,
    '중기업' : 1,
    '중소기업' : 1,
    '중견기업': 2,
    '대기업': 3,
}, inplace=True)

In [8]:
df.head()

,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros
0,서울,0,3,1,"5,342억 8,400만원 (2022)","1,251 명 재직중","6,218만원\n자세히보기",550억원,1000대-안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-매출 1000대 기...
1,서울,1,2,1,"5,342억 8,400만원 (2022)","1,251 명 재직중",-,550억원,1000대-KOSPI-복지의 끝판왕-매출 1조 클럽-보너스로 행복해요♥-집 걱정 안...
2,서울,4,3,3,"18조 3,714억 3,700만원 (2023)","19,824 명 재직중","7,656만원\n자세히보기","1조 5,644억 9,900만원",1000대-KT그룹-안정적인 장수 회사-구직자 인기기업-매출 1조 클럽-HOT한 인...
3,서울,4,3,3,"8,477억 5,653만원 (2022)",409 명 재직중,"7,177만원\n자세히보기","677억 1,334만원",1000대-한화그룹-성장판 열린 회사-구직자 인기기업-평균연봉 6천이상-지방에 위치...
4,서울,0,3,3,"8,477억 5,653만원 (2022)",409 명 재직중,-,"677억 1,334만원",KOSDAQ-구직자 인기기업-복지의 끝판왕-아침을 왜 굶어요?-자기계발비 지원 기업...


In [9]:
for col in ['sales', 'aversalary', 'capital']:
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].str.replace('조 ', '')
    df[col] = df[col].str.replace('억 ', '')

for col in ['sales', 'aversalary', 'capital']:
    df[col] = df[col].str.replace('만원', '')
    df[col] = df[col].str.replace('억원', '0000')
    df[col] = df[col].str.replace('조원', '00000000')

# run this block only once
df['sales'] = df['sales'].str[:-7]
df['aversalary'] = df['aversalary'].str.replace('\n자세히보기', '')
df['employees'] = df['employees'].str.replace(' 명 재직중', '')

df['pros'] 분석

In [12]:
from selenium import webdriver
# The By class is used to locate elements within a document.
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait

options = webdriver.ChromeOptions()
# Do not use GUI
options.add_argument("headless")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
# debug console log
options.add_argument("--log-level=3")
# To enable .click() in headless mode
options.add_argument("--window-size=1024,800")


with webdriver.Chrome(options=options) as driver:
    df = pd.DataFrame()

    for i in range(1, 34):
        driver.get("https://job.incruit.com/jobdb_list/categoryjob.asp?compcate={:02d}".format(i))
        
        tag = driver.find_element(By.XPATH, "//*[@id=\"incruit_contents\"]/div[1]/div/div[1]/h2").text
        print("{} : {}".format(i, tag))
        df = pd.concat([df, pd.DataFrame.from_dict([tag])], ignore_index=True)
        
    df.to_csv("tags.csv", encoding="utf-8-sig")


1 : 안정적인 장수 회사
2 : 성장판 열린 회사
3 : 구직자 인기기업
4 : 채용에 적극적인 회사
5 : 복지의 끝판왕
6 : 매출 1조 클럽
7 : 매출 상위 10%
8 : HOT한 인기기업
9 : 아침을 왜 굶어요?
10 : 평균연봉 6천이상
11 : 신입연봉이 4천이상
12 : 스타트업 초봉 3,500만원↗
13 : 지방에 위치한 대기업
14 : 자기계발비 지원 기업
15 : 보너스로 행복해요♥
16 : 내 출퇴근은 내 맘대로
17 : 아낌없이 주는 커피
18 : 아이와 함께 출근해요
19 : 잘나가는 의료·바이오기업
20 : 주 4일 근무
21 : 집 걱정 안 하기
22 : 거침없이 성장하는 회사
23 : 매출 1000대 기업
24 : 인크루트 인증 우수기업
25 : 대기업 in 서울
26 : 공기업 in 서울
27 : 장기근속수당제공 기업
28 : 육아휴직 제공 기업
29 : KOSPI 상장
30 : KOSDAQ 상장
31 : 빵빵한 구내식당
32 : 휴식까지 챙겨줘요!
33 : KONEX 상장


In [10]:
df.to_csv('raw_data2.csv', encoding='utf-8-sig')